# COVID-19 en Centroamérica
> Dashboard

- toc: false
- branch: master
- badges: true
- comments: false
- image: images/covid-19-ca.png
- permalink: /covid-19-ca/
- author: "Libre AI"
- categories: [centroamerica, covid19]

In [233]:
#hide
import numpy as np
import math
import folium
import pandas as pd
import flag
import altair as alt

from IPython.display import display, display_html, HTML, IFrame, Markdown

In [234]:
#hide_input

from datetime import datetime
utc_now = datetime.utcnow()

Markdown('> Última actualización: {} UTC'.format(utc_now.strftime("%Y-%m-%d %H:%M")))

> Última actualización: 2020-04-25 18:54 UTC

In [235]:
#hide

#latest case count
country_code2name = {'gt': 'Guatemala',
                     'bz': 'Belize',
                     'hn': 'Honduras',
                     'sv': 'El Salvador',
                     'ni': 'Nicaragua',
                     'cr': 'Costa Rica',
                     'pa': 'Panama'
                    }



In [236]:
#hide

population_in_m = {'gt': 16.6,
                   'bz': 0.41,
                   'hn': 9.16,
                   'sv': 6.49,
                   'ni': 6.46,
                   'cr': 5.06,
                   'pa': 4.22
                  }

In [237]:
#hide
confirmed = pd.read_csv('data/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('data/time_series_covid19_deaths_global.csv')
recovered = pd.read_csv('data/time_series_covid19_recovered_global.csv')

In [238]:
#hide

def get_latest_count(df, country):
    df_country = df[df['Country/Region'] == country]
    # dates are columns, so we take the last column
    latest_count_date = df_country.columns[-1]
    latest_count = df_country[latest_count_date]
    return latest_count.values[0]

def dmy_to_isodate(dmy):
    m, d, y = dmy.split('/')
    return '{}-{:02d}-{:02d}'.format(2000 + int(y), int(m), int(d))
    
def get_count_data(df_all, country, start_count):
    df = df_all[df_all['Country/Region'] == country]
    counts = list(enumerate([(dmy_to_isodate(x[0]), x[1]) for x in list(zip(df.columns[4:], list(df.values[0][4:]))) if x[1] >= start_count]))
    days_since = [x for x,(_, _) in counts]
    dates = [x for _,(x, _)  in counts]
    freqs = [x for _,(_,x) in counts]
    return dates, freqs, days_since

def get_count_df(df_all, country, start_count, metric, normalization_constant=1):
    dates, freqs, days_since = get_count_data(df_all, country, start_count)
    norm_freqs = [math.ceil(x/normalization_constant) for x in freqs]
    x = pd.DataFrame({
        'date': dates,
        'country': country,
        metric: norm_freqs,
        'new_{}'.format(metric): np.hstack((np.array([0]), np.diff(norm_freqs))),
        'days_since_{}_{}'.format(start_count, metric): days_since
    })
    return x
    
def get_confirmed_df(df_all, country, start_count=10):
    return get_count_df(df_all, country, start_count, 'cases')

def get_confirmed_per_m_df(df_all, country, start_count=1, normalization_constant=1):
    return get_count_df(df_all, country, start_count, 'cases_per_million', normalization_constant=normalization_constant)

def get_deaths_df(df_all, country, start_count=0):
    return get_count_df(df_all, country, start_count, 'deaths')

def get_recovered_df(df_all, country, start_count=0):
    return get_count_df(df_all, country, start_count, 'recovered')
    
def get_metrics_for_country(confirmed_all, deaths_all, recovered_all, country, normalization_constant=1):
    cases = get_confirmed_df(confirmed_all, country, start_count=1)
    cases_per_m = get_confirmed_per_m_df(confirmed_all, country, start_count=1, normalization_constant=normalization_constant)
    deaths = get_deaths_df(deaths_all, country, start_count=0)
    recovered = get_recovered_df(recovered_all, country, start_count=0)
    
    c = pd.merge(cases, cases_per_m, on=['date', 'country'], how='left')
    cd = pd.merge(c, deaths, on=['date', 'country'], how='left')
    cdr = pd.merge(cd, recovered, on=['date', 'country'], how='left')
    
    return cdr
    
    
    

In [239]:
#hide 
latest_cases_by_country = {c_code:get_latest_count(confirmed, country_code2name[c_code]) for c_code in country_code2name}

latest_deaths_by_country = {c_code:get_latest_count(deaths, country_code2name[c_code]) for c_code in country_code2name}

latest_recovered_by_country = {c_code:get_latest_count(recovered, country_code2name[c_code]) for c_code in country_code2name}

('latest_cases_by_country', latest_cases_by_country,
 'latest_deaths_by_country', latest_deaths_by_country,
 'latest_recovered_by_country', latest_recovered_by_country)

('latest_cases_by_country',
 {'gt': 430, 'bz': 18, 'hn': 591, 'sv': 274, 'ni': 11, 'cr': 687, 'pa': 5338},
 'latest_deaths_by_country',
 {'gt': 11, 'bz': 2, 'hn': 55, 'sv': 8, 'ni': 3, 'cr': 6, 'pa': 154},
 'latest_recovered_by_country',
 {'gt': 30, 'bz': 5, 'hn': 58, 'sv': 75, 'ni': 7, 'cr': 216, 'pa': 319})

In [240]:
#hide
#The Map ---

In [241]:
#hide

map_center_lat, map_center_long = (12.5, -87)
zoom = 5

In [242]:
#hide

class Country:
    def __init__(self, name, lat, long):
        super()
        self.cases = 0
        self.deaths = 0
        self.recovered = 0
        self.name = name
        self.lat = lat
        self.long = long
        self.population_in_m = 1


In [243]:
#hide

gt = Country(country_code2name['gt'], 14.63, -90.56)
bz = Country(country_code2name['bz'], 17.25, -88.80)
hn = Country(country_code2name['hn'], 14.08, -87.24)
sv = Country(country_code2name['sv'], 13.69, -89.25)
ni = Country(country_code2name['ni'], 12.10, -86.33)
cr = Country(country_code2name['cr'], 9.94, -84.15)
pa = Country(country_code2name['pa'], 9.08, -79.59)

countries = {'gt': gt, 'bz': bz, 'hn': hn, 'sv': sv, 'ni': ni, 'cr': cr, 'pa': pa}

In [244]:
#hide

# set latest cases
for c in countries:
    countries[c].cases = latest_cases_by_country[c]
    countries[c].deaths = latest_deaths_by_country[c]
    countries[c].recovered = latest_recovered_by_country[c]
    countries[c].population_in_m = population_in_m[c]
        

In [245]:
#hide
# confirmed, deaths, recovered (cdr) for each country
cdr = {}
for c in countries:
    cdr[c] = get_metrics_for_country(confirmed, deaths, recovered, country_code2name[c], population_in_m[c])
    
cdr_ca = None
for c in countries:
    if cdr_ca is None:
        cdr_ca = cdr[c]
    else:
        cdr_ca = cdr_ca.append(cdr[c])
        

In [246]:
#hide
cdr['sv'].tail(1)

,date,country,cases,new_cases,days_since_1_cases,cases_per_million,new_cases_per_million,days_since_1_cases_per_million,deaths,new_deaths,days_since_0_deaths,recovered,new_recovered,days_since_0_recovered
36,2020-04-24,El Salvador,274,24,36,43,4,36,8,0,93,75,8,93


In [247]:
#hide

cdr_ca[cdr_ca['country'] == country_code2name['sv']].head()

,date,country,cases,new_cases,days_since_1_cases,cases_per_million,new_cases_per_million,days_since_1_cases_per_million,deaths,new_deaths,days_since_0_deaths,recovered,new_recovered,days_since_0_recovered
0,2020-03-19,El Salvador,1,0,0,1,0,0,0,0,57,0,0,57
1,2020-03-20,El Salvador,1,0,1,1,0,1,0,0,58,0,0,58
2,2020-03-21,El Salvador,3,2,2,1,0,2,0,0,59,0,0,59
3,2020-03-22,El Salvador,3,0,3,1,0,3,0,0,60,0,0,60
4,2020-03-23,El Salvador,3,0,4,1,0,4,0,0,61,0,0,61


In [248]:
#hide

m = folium.Map(location=[map_center_lat, map_center_long],
               zoom_start=zoom,
               max_zoom=zoom+1,
               min_zoom=zoom-1
              )

# tiles = 'cartodbdark_matter'
# tiles = 'cartodbpositron'
# tiles = 'stamentoner'
tiles = 'stamenterrain'

folium.TileLayer(tiles, min_zoom=zoom-1, max_zoom=zoom+1).add_to(m)

tooltip_template = '''<div>
<b>{}</b><br/>
<span style='color:orange'><h3 style='display:inline'>{}</h3></span>&nbsp;Diagnosticados<br/>
<span style='color:red'><h4 style='display:inline'>{}</h4></span>&nbsp;Muertos<br/>
<span style='color:green'><h4 style='display:inline'>{}</h4></span>&nbsp;Curados
</div>
'''

for c in countries:
    folium.CircleMarker(
        location=[countries[c].lat, countries[c].long],
        radius=math.sqrt(countries[c].cases),
        color='red',
        fill=True,
        fill_color='#f03',
        tooltip=tooltip_template.format(countries[c].name,
                                        countries[c].cases,
                                        countries[c].deaths,
                                        countries[c].recovered
                                       )
    ).add_to(m)


m.save('../maps/covid19-ca-map.html')

In [249]:
# hide_input
display(IFrame('../maps/covid19-ca-map.html', height=400, width=500))

---

In [250]:
#hide
# The table ---

In [251]:
#hide

def get_summary_table():
    country_code_name = sorted(country_code2name.items(), key=lambda x: x[1])
    country_code = [x[0] for x in country_code_name]
    country_name = ["{} {}".format(x[1], flag.flag(x[0])) for x in country_code_name]

    cases_per_country = []
    deaths_per_country = []
    recovered_per_country = []

    for c in country_code:
        cases_per_country.append(latest_cases_by_country[c])
        deaths_per_country.append(latest_deaths_by_country[c])
        recovered_per_country.append(latest_recovered_by_country[c])
    
    x = pd.DataFrame({'País': country_name,
                      'Diagnosticados': cases_per_country,
                      'Muertos': deaths_per_country,
                      'Curados': recovered_per_country
                     })
    return x



In [252]:
#hide_input

summary_df = get_summary_table()

heading_properties = [('font-size', '14px')]
cell_properties = [('font-size', '14px')]
dfstyle = [dict(selector="th", props=heading_properties), 
           dict(selector="td", props=cell_properties)]

df_style = summary_df.style\
.hide_index()\
.background_gradient(cmap='Oranges', subset=['Diagnosticados'])\
.background_gradient(cmap='Reds', subset=['Muertos'])\
.background_gradient(cmap='Greens', subset=['Curados'])\
.set_table_styles(dfstyle)

# display(HTML('<div style="display: flex; justify-content: center;">{}</div>'.format(df_style.render())))
display(HTML('{}'.format(df_style.render())))



País,Diagnosticados,Muertos,Curados
Belize 🇧🇿,18,2,5
Costa Rica 🇨🇷,687,6,216
El Salvador 🇸🇻,274,8,75
Guatemala 🇬🇹,430,11,30
Honduras 🇭🇳,591,55,58
Nicaragua 🇳🇮,11,3,7
Panama 🇵🇦,5338,154,319


---

In [253]:
#hide
# For the cases per million plot ---


In [254]:
#hide
cdr_ca.columns

Index(['date', 'country', 'cases', 'new_cases', 'days_since_1_cases',
       'cases_per_million', 'new_cases_per_million',
       'days_since_1_cases_per_million', 'deaths', 'new_deaths',
       'days_since_0_deaths', 'recovered', 'new_recovered',
       'days_since_0_recovered'],
      dtype='object')

## Diagnosticados por Millon de Habitantes

In [255]:
#hide_input

highlight = alt.selection(type='single',
                          on='mouseover',
                          fields=['country'],
                          nearest=True)

base = alt.Chart(cdr_ca, title="").encode(
    x=alt.X('days_since_1_cases_per_million:Q', title="días desde el primer caso por millon"),
    y=alt.Y('cases_per_million:Q', scale=alt.Scale(type='log'), title="diagnosticados por millon (log-scale)"),
    color=alt.Color('country:N', title="País"),
    tooltip=[alt.Tooltip('country', title='país'),
             alt.Tooltip('date', title='fecha'),
             alt.Tooltip('cases_per_million:Q', title='Diagnosticados por millon'),
             alt.Tooltip('cases:Q', title='Diagnosticados (Total)'),
             alt.Tooltip('new_cases:Q', title='Diagnosticados (Nuevos)'),
             alt.Tooltip('deaths:Q', title='Muertos (Total)'),
             alt.Tooltip('new_deaths:Q', title='Muertos (Nuevos)')
            ]
)

points = base.mark_circle().encode(
    size=alt.Size('new_cases:Q', title="Nuevos casos"),
).add_selection(
    highlight
)

lines = base.mark_line(opacity=0.5).encode(
    size=alt.condition(~highlight, alt.value(2.5), alt.value(5)),
    
)



alt.layer(points, lines).properties(
    height=600,
    width=600
).configure_axis(
    labelFontSize=18,
    titleFontSize=18
)

alt.LayerChart(...)

---

## Fuentes
- 2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE [https://github.com/CSSEGISandData/COVID-19]
- Población Mundial por País [https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population]

---

<div align="right">Notebook and graphics by Dr. Ernesto Diaz-Aviles, <a href="https://twitter.com/vedax" target="_blank">@vedax</a><br/><a href="https://libreai.com" target="_blank">Libre AI</a><br/><a href="https://creativecommons.org/licenses/by-sa/4.0/" target="_blank">CC BY-SA 4.0</a>, 2020</div>

---